In [4]:
#!pip install yfinance
#!pip install backtesting

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
%matplotlib inline

In [21]:
from backtesting import Backtest
from backtesting.test import EURUSD, GOOG
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import matplotlib.pyplot as plt

In [22]:
data = yf.download('MSFT', start='2024-01-01', end='2024-05-06')

[*********************100%%**********************]  1 of 1 completed


In [23]:
class SMACrossoverWithSLTP(Strategy):
    short_sma_period = 10
    long_sma_period = 25

    def init(self):
        self.short_sma = self.I(SMA, self.data.Close, self.short_sma_period)
        self.long_sma = self.I(SMA, self.data.Close, self.long_sma_period)

    def next(self):
        current_price = self.data.Close[-1]
        long_stop_loss = current_price * 0.99
        long_take_profit = current_price * 1.05
        short_stop_loss = current_price * 1.01
        short_take_profit = current_price * 0.95

        if crossover(self.short_sma, self.long_sma):
            if self.position.is_short:
                self.position.close()
            self.buy(size=0.5, sl=long_stop_loss, tp=long_take_profit)
        elif crossover(self.long_sma, self.short_sma):
            if self.position.is_long:
                self.position.close()
            self.sell(size=0.5, sl=short_stop_loss, tp=short_take_profit)

In [28]:
backtest_sma_sl_tp = Backtest(data, SMACrossoverWithSLTP, cash=10000, commission=0.002, exclusive_orders=True)
results_sma_sl_tp = backtest_sma_sl_tp.run()
backtest_sma_sl_tp.plot()
print(results_sma_sl_tp)


c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = grid

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    33.72093
Equity Final [$]                  9750.012919
Equity Peak [$]                       10000.0
Return [%]                          -2.499871
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                   -7.149841
Volatility (Ann.) [%]                 3.94455
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.642838
Avg. Drawdown [%]                   -2.642838
Max. Drawdown Duration       66 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                    5
Win Rate [%]                              0.0
Best Trade [%]                      -0.554803
Worst Trade [%]                     -1.819001
Avg. Trade [%]                    

In [14]:
optimized_sma_sl_tp = backtest_sma_sl_tp.optimize(short_sma_period=range(5, 30, 5), long_sma_period=range(15, 50, 5), maximize='Sharpe Ratio')
print(optimized_sma_sl_tp)
print(optimized_sma_sl_tp._strategy)

c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    1.162791
Equity Final [$]                 10055.619293
Equity Peak [$]                  10055.619293
Return [%]                           0.556193
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                    1.638538
Volatility (Ann.) [%]                0.973322
Sharpe Ratio                         1.683449
Sortino Ratio                             inf
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       1.178649
Worst Trade [%]                      1.178649
Avg. Trade [%]                    

c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [15]:
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [16]:
data['RSI'] = compute_rsi(data['Close'])

In [17]:
class RSIStrategy(Strategy):
    rsi_period = 14

    def init(self):
        self.rsi = self.data.RSI

    def next(self):
        if self.rsi[-1] < 30:
            self.buy()
        elif self.rsi[-1] > 70:
            self.position.close()

In [18]:
backtest_rsi = Backtest(data, RSIStrategy, cash=10000, commission=0.002, exclusive_orders=True)
results_rsi = backtest_rsi.run()
backtest_rsi.plot()
print(results_rsi)

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = grid

In [19]:
optimized_rsi = backtest_rsi.optimize(rsi_period=range(10, 35, 5), maximize='Sharpe Ratio')
print(optimized_rsi)
print(optimized_rsi._strategy)

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

c:\Users\dellek\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\backtesting.py:1384: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  best_params = heatmap.idxmax()
